In [93]:
# ------------------------------
# Imports & setup
# ------------------------------
import nest_asyncio
nest_asyncio.apply()

import requests
import pandas as pd
import json
import re
import numpy as np
import asyncio
import os
from dotenv import load_dotenv

from google.adk import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
model_name = os.getenv("MODEL") or "gemini-2.5-pro"

In [94]:
# ------------------------------
# Yahoo Finance helpers
# ------------------------------
URL_BASE = "https://query1.finance.yahoo.com/v1/finance/screener/predefined/saved?scrIds={}&count={}"

def fetch_screener(scrId, count=250):
    url = URL_BASE.format(scrId, count)
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    data = r.json()["finance"]["result"][0]["quotes"]
    return pd.DataFrame(data)

def fetch_trending():
    url = "https://query1.finance.yahoo.com/v1/finance/trending/US"
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    data = r.json()["finance"]["result"][0]["quotes"]
    return pd.DataFrame(data)

def filter_equities(df):
    eq = []
    for s in df["symbol"]:
        if re.fullmatch(r"[A-Za-z][A-Za-z0-9\.-]{0,9}", s):
            eq.append(s)
    return pd.DataFrame({"symbol": eq})

def get_day_gainers():
    df = fetch_screener("day_gainers")
    return df.head(20)

def get_most_actives():
    df = fetch_screener("most_actives")
    return df.head(20)

def get_trending_tickers():
    df_raw = fetch_trending()
    df = filter_equities(df_raw)
    return df.head(20)

# Convert DataFrame to JSON
def df_to_json(df: pd.DataFrame) -> str:
    df_clean = df.replace({np.nan: None})
    return df_clean.to_json(orient="records")

In [95]:
# ------------------------------
# Setup AI agent
# ------------------------------
user_id = "user_app"
app_name = "finance_app"

finance_agent = Agent(
    model=model_name,
    name="finance_agent",
    instruction=(
        "You are a finance assistant. "
        "Decide whether to call get_day_gainers, get_most_actives, or get_trending_tickers "
        "based on the user's request."
    ),
    tools=[],  # leave empty for now, ADK cannot call plain Python functions directly
)

runner = InMemoryRunner(agent=finance_agent, app_name=app_name)

# Create session
session = await runner.session_service.create_session(
    app_name=app_name,
    user_id=user_id
)
print("Session created:", session.id)

Session created: 74b83dda-3bb9-4fda-9fcf-d3a71468bc51


In [102]:
async def ask_agent(prompt: str, df_json: str = None) -> str:
    """
    Send prompt to agent. Optionally include JSON data for analysis.
    Returns the agent's text output as a string.
    """
    full_prompt = prompt
    if df_json:
        full_prompt += f"\n\nStock Data:\n{df_json}"

    part = types.Part()
    part.text = full_prompt

    content = types.Content(
        role="user",
        parts=[part]
    )

    # Collect all text parts
    result_text = ""
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session.id,
        new_message=content
    ):
        for part in event.content.parts:
            if hasattr(part, "text") and part.text:
                print("Finance →", part.text)  # still print
                result_text += part.text

    return result_text.strip()


In [100]:
# ------------------------------
# Main workflow
# ------------------------------
async def ask_finance(question: str):
    # Step 1: Ask agent which tool to use
    decision_prompt = (
        f"QUESTION: {question}\n"
        "Decide which tool to use: get_day_gainers, get_most_actives, or get_trending_tickers. "
        "Respond with the exact tool name only."
    )
    tool_name = await ask_agent(decision_prompt)

    # Step 2: Execute selected tool
    tool_map = {
        "get_day_gainers": get_day_gainers,
        "get_most_actives": get_most_actives,
        "get_trending_tickers": get_trending_tickers
    }

    if tool_name.strip() in tool_map:
        df = tool_map[tool_name.strip()]()
        df_json = df_to_json(df)

        # Step 3: Ask agent to analyze the data
        analysis_prompt = f"Analyze this stock data and summarize the key insights."
        await ask_agent(analysis_prompt, df_json)
    else:
        print("Agent did not return a valid tool. Try rephrasing the question.")

In [103]:
await ask_finance("Show me today's top stock gainers")

Finance → get_day_gainers
Finance → Based on the stock data provided, here is a summary of the key insights:

Today's market saw significant gains across several stocks, with many experiencing double-digit percentage increases on high trading volume.

**Top Performers:**

*   **Arrowhead Pharmaceuticals (ARWR):** Was the standout performer, surging **23.34%** to close at $57.71. The stock's volume was nearly triple its three-month average, and it hit a new 52-week high of $59.15 during the trading day.
*   **CleanSpark, Inc. (CLSK):** Increased by **13.79%** to $13.45 on exceptionally high trading volume of nearly 70 million shares. The stock has a "Strong Buy" average analyst rating.
*   **Urban Outfitters, Inc. (URBN):** Gained **13.54%** to finish at $77.56, with trading volume almost five times its daily average.

**Key Trends & Observations:**

*   **High Volume Movers:** Almost all the top-gaining stocks traded on significantly higher-than-average volume, indicating strong invest

In [104]:

await ask_finance("Which stocks are trending in the market today?")

Finance → get_trending_tickers
Finance → Based on the list of trending tickers, here is a summary of the key market insights:

Today's market interest is concentrated on a diverse mix of assets, highlighting several dominant themes:

*   **Cryptocurrency and Digital Assets:** There is significant attention on major cryptocurrencies like **Ethereum (ETH-USD)** and **XRP (XRP-USD)**. This interest extends to the broader crypto ecosystem, including crypto-mining stocks like **Bitmine Immersion (BMNR)** and **Iris Energy (IREN)**.

*   **Major Technology and AI Leaders:** Well-known tech giants are trending, with a particular focus on companies central to the AI narrative, such as **NVIDIA (NVDA)**, **Meta Platforms (META)**, **Alphabet (GOOGL)**, and **Broadcom (AVGO)**. **Tesla (TSLA)** also continues to be a focal point of investor attention.

*   **Retail and "Meme" Stock Interest:** The presence of **GameStop (GME)** indicates continued engagement from retail investors in stocks known

In [ ]:

await ask_finance("Give me the most actively traded stocks")